In [1]:
from dxfwrite import DXFEngine as dxf
import numpy as np
import matplotlib.pyplot as plt
#import microfluidics_class as mf
from copy import deepcopy

from microfluidics_class import Design
from microfluidics_class import Feature

In [2]:
design = Design()
design.file = '/Users/guillaume/Desktop/test_class3.dxf'

In [3]:
design.add_layer('Layer1', {'name':'first_layer','color':1, 'inversion':0})
design.add_layer('Layer2', {'name':'sec_layer','color':2, 'inversion':0})
design.add_layer('Layer3', {'name':'third_layer','color':3, 'inversion':0})
design.add_layer('Layer4', {'name':'fourth_layer','color':4, 'inversion':0})

In [5]:
tube_coord = [[0,0],[1000,0],[1000,1000]]
tube_rad = 20
tube_curv = [0,50,0]

In [11]:
myfeature1 = Feature('Layer1').define_tube(tube_coord,tube_curv,tube_rad)

In [14]:
Feature('Layer1').layer

'Layer1'

In [6]:
myfeature1 = Feature.define_tube(tube_coord,tube_curv,tube_rad)
myfeature1.set_layer('Layer1')
myfeature1.set_mirror(0)

In [7]:
myfeature2 = Feature.circular_punching(nb_points=20,outer_rad=100,position=[0,0]).layer('Layer2')

TypeError: 'NoneType' object is not callable

In [29]:
myfeature2 = Feature.circular_punching(nb_points=20,outer_rad=100,position=[0,0]).set_layer('Layer2')

In [30]:
#copy a given feature and move it
myfeature2_copy = myfeature2.copy().move([1000,2000])

In [31]:
design.add_feature('myfirstfeature', myfeature1)
design.add_feature('mysecondfeature', myfeature2)
design.add_feature('mysecondfeature_copy', myfeature2_copy)

In [95]:
myfeature3 = Feature.channel_array(length=100,num=2,origin=[0,-1000],space=200,space_series=200,subsampling=1,widths=[1,2]).set_layer('Layer1')
numfeature = Feature.numbering(num=18.6,pos=[2000,1000],rotation=np.pi/4,scale=10).set_layer('Layer1')
numarray = Feature.number_array(num_series=2,origin=[-2000,-1000],num=3,space=100,space_series=200,rotation=0,scale=5,subsampling=1).set_layer('Layer1')
pattern = Feature.patterned_region(channel_separation=10,channel_width=1,global_shape=[[200,200],[400,200],[400,400],[200,400]])
pattern.set_layer('Layer1')
align1, align2 = Feature.align_mark_squares(pos=[300,600])
align1.set_layer('Layer1')
align2.set_layer('Layer2')
combined = Feature.combine_features(align1,align2).set_layer('Layer3')
inverted = Feature.reverse_feature(numarray,[[-2000,-1050],[-1000,-1050],[-1000,-950],[-2000,-950]])
inverted.set_layer('Layer3')

In [96]:
design.add_feature('ch_array', myfeature3)
design.add_feature('numfeature', numfeature)
design.add_feature('numarray', numarray)
design.add_feature('pattern', pattern)
design.add_feature('align1', align1)
design.add_feature('align2', align2)
design.add_feature('combined', combined)
design.add_feature('inverted', inverted)

In [97]:
#copy design at multiple positions
fulldesign = design.multiplicate([[0,1000],[0,-1000]])

#draw full design
fulldesign.draw_design()

#close the drawing
fulldesign.close()

In [98]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
from ipywidgets import SelectMultiple
from ipywidgets import interactive
from ipywidgets import interact
from ipywidgets import Checkbox
from bokeh.models import Range1d

Loading BokehJS ...

In [99]:
sel_mult = SelectMultiple(options = [x for x in design.layers])

In [120]:
def g(m):

    layers_to_plot = m
    if len(layers_to_plot)>0:
        features_to_plot = [x for x in design.features if design.features[x].layer in layers_to_plot]
        if len(features_to_plot)>0:
            #chk = [Checkbox(description=a) for a in features_to_plot]
            #interact(updatePlot, **{c.description: c.value for c in chk})
            sel_mult2 = SelectMultiple(options = features_to_plot)
            interactive_plot2 = interactive(updatePlot, n=sel_mult2)
            display(interactive_plot2)
            
def find_min(cur_min, new_min):
    if new_min<cur_min:
        cur_min = new_min
    return cur_min

def find_max(cur_max, new_max):
    if new_max>cur_max:
        cur_max = new_max
    return cur_max

def update_bounds(cur_bound, feature_coord):
    cur_bound[0] = find_min(cur_bound[0],np.min([np.min(x[:,0]) for x in feature_coord]))
    cur_bound[1] = find_max(cur_bound[1],np.max([np.max(x[:,0]) for x in feature_coord]))

    cur_bound[2] = find_min(cur_bound[2],np.min([np.min(x[:,1]) for x in feature_coord]))
    cur_bound[3] = find_max(cur_bound[3],np.max([np.max(x[:,1]) for x in feature_coord]))

    return cur_bound


def updatePlot(n):
    #clear_output()
    features_to_plot = n
    if len(features_to_plot)>0:
        p = figure(title="simple line example", x_axis_label='x', y_axis_label='y',output_backend="webgl")
        cur_bound = [100000,-100000,100000,-100000]
        
        for elem in features_to_plot:
            toplot = design.features[elem].coord
            cur_bound = update_bounds(cur_bound, toplot)
            for tp in toplot:
                p.line(tp[:,0], tp[:,1])
            if not design.features[elem].mirror == None:
                flipped = design.features[elem].flip_feature(0).coord
                cur_bound = update_bounds(cur_bound, flipped)
                for tp in flipped:
                    p.line(tp[:,0], tp[:,1])
                    
        maxrange = np.max([cur_bound[1]-cur_bound[0],cur_bound[3]-cur_bound[2]])
        p.x_range=Range1d(cur_bound[0],cur_bound[0]+maxrange)
        p.y_range=Range1d(cur_bound[2],cur_bound[2]+maxrange)
        
        #p.x_range=Range1d(np.min([cur_bound[0],cur_bound[2]]),np.max([cur_bound[1],cur_bound[3]]))
        #p.y_range=Range1d(np.min([cur_bound[0],cur_bound[2]]),np.max([cur_bound[1],cur_bound[3]]))
        
        plot_height = 800
        plot_width = 800
        
        #plot_width = int(800*((cur_bound[1]-cur_bound[0])/(cur_bound[3]-cur_bound[2])))
        #plot_height = 800
        #if plot_width>800:
        #    p.plot_height = int(plot_height/(plot_width/800))
        #    p.plot_width = 800
        #elif plot_height>800:
        #    p.plot_width = int(plot_height/(plot_height/800))
        #    p.plot_height = 800
        #else:
        #    p.plot_width = plot_width
        #    p.plot_height = plot_height
        
        #p.plot_height = 100
        #p.inner_height = 100
        #p.inner_width  = 100
        
        show(p)

In [121]:
interactive_plot = interactive(g, m=sel_mult)
#interactive_plot.layout.height = '200'
#interactive_plot.layout.width = '100'
display(interactive_plot)

A Jupyter Widget

In [109]:
p3.

In [2]:
p3 = figure(title="simple line example", x_axis_label='x', y_axis_label='y')
p3.line([1,2,3],[1,2,3])

GlyphRenderer(id='69140a28-d91a-412b-9a4f-8f88cdc34622', ...)

In [7]:
p3.width =100
p3.height =100

In [8]:
show(p3)

In [15]:
truc = p3.x_range

In [21]:
truc.bounds[0]=0

TypeError: 'NoneType' object does not support item assignment

In [126]:
4*np.ones([3,3])

array([[ 4.,  4.,  4.],
       [ 4.,  4.,  4.],
       [ 4.,  4.,  4.]])